In [3]:
import torch
from transformers import XLNetTokenizer, XLNetLMHeadModel

In [4]:
model = XLNetLMHeadModel.from_pretrained("xlnet-base-cased")
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Downloading: 100%|██████████| 760/760 [00:00<00:00, 762kB/s]
Downloading: 100%|██████████| 467M/467M [00:12<00:00, 38.0MB/s]
Downloading: 100%|██████████| 798k/798k [00:00<00:00, 5.06MB/s]


In [133]:
origStr = "I mean, when you go to a movie and it’s set to start at a certain time, would you not be upset if 7 hours later said movie has not started?"
testStr = "<mask> <mask> <mask> <mask> <mask> <mask> a <mask> and <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask>?"
encoded_str = tokenizer.encode(testStr)
tokens_tensor = torch.tensor([encoded_str])
tokenizer.convert_ids_to_tokens(encoded_str)
perm_mask = torch.zeros((1, tokens_tensor.shape[1], tokens_tensor.shape[1]), dtype=torch.float)
perm_mask[:, :, 8] = 1.0  # Previous tokens don't see masked token
target_mapping = torch.zeros((1, 1, tokens_tensor.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
target_mapping[0, 0, 8] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)


In [134]:
with torch.no_grad():
    outputs = model(tokens_tensor, perm_mask=perm_mask, target_mapping=target_mapping)
    next_token_logits = outputs[0][0, 0, :]

In [135]:
print(outputs[0].shape)

torch.Size([1, 1, 32000])


In [136]:
print([tokenizer.convert_ids_to_tokens(index.item()) for index in next_token_logits.topk(10).indices])

['.', 'd', '▁a', '?', '-', '▁A', 'wo', ',', '▁and', 'ly']
